In [1]:
import numpy as np
from numba import njit
from scipy.stats.mstats import mquantiles 
import concurrent.futures as fut

# Optimizing Numpy Performance - Example VaR Calculation

## VaR

In [2]:
pl = np.random.rand(5000, 150)

In [3]:
cl = 1 - 0.95

In [4]:
def quan(pl):
    return mquantiles(pl, cl, alphap=0, betap=0, axis=1)

In [5]:
%timeit var = quan(pl)

419 ms ± 66.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
var = quan(pl)

In [7]:
var.shape

(5000, 1)

In [8]:
%%time
pl_gen = (np.random.rand(5000, 150) for i in range(10))
res = [quan(pl) for pl in pl_gen]

Wall time: 3.84 s


## Subwindow VaR

### Sinlge Trade

In [9]:
pl1 = pl[0]

In [10]:
window = 60

In [11]:
%%timeit
cba_var1 = []
for i in range(len(pl1) - window + 1):
    sw = pl1[i:i+window]
    cba_var1.append(mquantiles(sw, cl, 0, 0)[0])

4.04 ms ± 30.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Pure Python approach for creating subwindows

In [12]:
def rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

In [13]:
%timeit rolling_window(pl1, window)

8.29 µs ± 51.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [14]:
%timeit cba_var2 = quan(rolling_window(pl1, window))

6.78 ms ± 607 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Vectorized subwindow creation is actually slower, not faster here.

In [15]:
@njit
def calc_cba(pl, window):
    nr_subwindows = len(pl) - window + 1
    cba_pls = np.zeros((nr_subwindows, window))
    for i in range(nr_subwindows):
        cba_pls[i, :] = pl[i:i+window]
    return cba_pls

In [16]:
%timeit cba_pls = calc_cba(pl1, window)

The slowest run took 4.16 times longer than the fastest. This could mean that an intermediate result is being cached.
9 µs ± 6.7 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
cba_pls = calc_cba(pl1, window)

In [18]:
cba_pls.shape

(91, 60)

In [19]:
%timeit cba_var3 = quan(cba_pls)

7.26 ms ± 1.06 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


Subwindow creation with Numba is also not faster.

### All Trades

In [20]:
@njit
def calc_cba2(pls, window):
    nr_pl = pls.shape[0]
    nr_subwindows = len(pls[0, :]) - window + 1
    cba_pls = np.zeros((nr_pl * nr_subwindows, window))
    for j in range(nr_pl):
        pl = pls[j, :]
        for i in range(nr_subwindows):
            cba_pls[i + j*nr_subwindows, :] = pl[i:i+window]
    return cba_pls

In [42]:
%timeit cba_pls2 = calc_cba2(pl, window)

180 ms ± 4.75 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [43]:
cba_pls2 = calc_cba2(pl, window)

Creating subwindows is actually not the bottleneck, even pure Python. Using Numba gives a further speedup.

In [30]:
%time cba_var4 = quan(cba_pls2)

Wall time: 34.2 s


Quantile calculation using Scipy mquantiles takes a very long time for a large number of PL vectors.

In [23]:
cba_pls2.shape

(455000, 60)

Parallelization gives the expected speedup.

But could we get faster?

## Reimplementation of SciPy Quantile Function

In [24]:
%timeit cba_pls2.sort()

91.6 ms ± 2.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Sorting the array is actually quite fast. The remaining part of quantile calculation are essentially just vector operations, which should be fast, too.

In [33]:
def quantile(data: np.ndarray, p: float, alphap: float = 0, betap: float = 0) -> np.ndarray:
    """
    More performant (but a little less general) re-implementation of scipy.stats.mstats.mquantiles.
    Performance gain compared to mquantiles is about a factor of 200 !
    Works on numpy arrays (not masked arrays) and supports calculation of a single quantile only.
    :param data: 2d input array. The 1st axis contain different series, for which the quantile is calculated
    along the 2nd axis
    :param p: quantile (for VaR: 1 - confidence level)
    :param alphap: alpha parameter of quantile function
    :param betap: beta parameter of quantile function
    returns 1d array of quantiles
    """
    assert data.ndim == 2
    n = data.shape[1]
    assert n > 1
    data.sort()
    m = alphap + p * (1. - alphap - betap)
    aleph = (n * p + m)
    k = min(max(int(aleph), 1), n - 1)
    gamma = min(max(aleph - k, 0), 1)
    return (1. - gamma) * data[:, (k - 1)] + gamma * data[:, k]

In [40]:
%timeit cba_var6 = quantile(cba_pls2, cl)

111 ms ± 3.99 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [41]:
cba_var6 = quantile(cba_pls2, cl)

In [39]:
all(cba_var4.compressed() == cba_var6)

True

The new implementation is a factor of 200 faster while getting exactly the same results!